In [2]:
base_model_url = "meta-llama/Llama-3.2-3B-Instruct"
new_model_url = "./llama-3.2-3b-it-Ecommerce-ChatBot/"

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
from peft import PeftModel
import torch
from trl import setup_chat_format
# Reload tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(base_model_url)

base_model_reload= AutoModelForCausalLM.from_pretrained(
    base_model_url,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

# Add a new padding token to the tokenizer
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# Resize the model embeddings to match the new tokenizer size
base_model_reload.resize_token_embeddings(len(tokenizer))

Loading checkpoint shards: 100%|██████████| 2/2 [00:08<00:00,  4.05s/it]
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(128257, 3072)

In [8]:
# Merge adapter with base model
if getattr(tokenizer, 'chat_template', None) is None:
    base_model_reload, tokenizer = setup_chat_format(base_model_reload, tokenizer)

model = PeftModel.from_pretrained(base_model_reload, new_model_url)

model = model.merge_and_unload()

In [10]:
instruction = """You are a top-rated customer service agent named John. 
    Be polite to customers and answer all their questions.
    """

messages = [{"role": "system", "content": instruction},
    {"role": "user", "content": "I have to see what payment payment modalities are accepted"}]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=150, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.




Thank you for reaching out to us! I understand your curiosity about the payment methods we accept. Allow me to provide you with a comprehensive list of the payment modalities we offer. We strive to offer flexibility and convenience to our customers, so you can choose the method that suits your needs best. Here are the payment options we accept:

- Credit/Debit Card: We accept major credit and debit cards such as Visa, Mastercard, and American Express.
- PayPal: A widely recognized and secure online payment method.
- Bank Transfer: You can make direct payments from your bank account.
- Apple Pay: For seamless payments using your Apple device.
- Google Pay: Another convenient option for Google users.

If you have any further questions or need assistance
